<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# TF-IDF Content-Based Recommendation


In [3]:
import sys
import logging
import scipy
import numpy as np
import pandas as pd
import dataset_loader

from recommenders.models.tfidf.tfidf_utils import TfidfRecommender
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.datasets import movielens
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, mae, rmse, novelty, historical_item_novelty, user_item_serendipity, user_serendipity, serendipity, catalog_coverage, distributional_coverage
from metrics import precision_at_k, recall_at_k, mrr, accuracy, user_coverage, item_coverage

# Print version
print(f"System version: {sys.version}")

c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


System version: 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]


### 1. Load the dataset into a dataframe
Let's begin by loading the metadata file for the dataset into a Pandas dataframe. This file contains metadata about each of the scientific articles included in the full dataset.

In [4]:
# Top k items to recommend
TOP_K = 10

In [5]:
data = dataset_loader.loader("movielens", want_col=["userID", "itemID", "rating", "timestamp", 'title', 'genres'], num_rows=500000)
print(data.shape)
print(data['userID'].nunique())

# Convert the float precision to 32-bit in order to reduce memory consumption
data["rating"] = data["rating"].astype(np.float32)
data.head()


(500000, 6)
3374


,userID,itemID,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


### 4. Instantiate the recommender
All functions for data preparation and recommendation are contained within the **TfidfRecommender** class we have imported. Prior to running these functions, we must create an object of this class.

Select one of the following tokenization methods to use in the model:

| tokenization_method | Description                                                                                                                      |
|:--------------------|:---------------------------------------------------------------------------------------------------------------------------------|
| 'none'              | No tokenization is applied. Each word is considered a token.                                                                     |
| 'nltk'              | Simple stemming is applied using NLTK.                                                                                           |
| 'bert'              | HuggingFace BERT word tokenization ('bert-base-cased') is applied.                                                               |
| 'scibert'           | SciBERT word tokenization ('allenai/scibert_scivocab_cased') is applied.<br>This is recommended for scientific journal articles. |

In [6]:
# Create the recommender object
recommender = TfidfRecommender(id_col='itemID', tokenization_method='bert')

### 5. Prepare text for use in the TF-IDF recommender

In [7]:
data['genres'] = data['genres'].str.replace('|', ' ', regex=False)
data.head()

,userID,itemID,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure Children Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure Drama Fantasy Mystery Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery Sci-Fi Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime Mystery Thriller


In [8]:
df_clean = data.drop(columns=['userID', 'rating', 'timestamp'])
df_clean = df_clean.drop_duplicates(subset=['itemID'])
cols_to_clean = ['title','genres']
clean_col = 'cleaned_text'
df_clean = recommender.clean_dataframe(df_clean, cols_to_clean, clean_col)
df_clean.head()

,itemID,title,genres,cleaned_text
0,2,Jumanji (1995),Adventure Children Fantasy,Jumanji 1995 Adventure Children Fantasy
1,29,"City of Lost Children, The (Cité des enfants p...",Adventure Drama Fantasy Mystery Sci-Fi,City of Lost Children The Cité des enfants per...
2,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery Sci-Fi Thriller,Twelve Monkeys aka 12 Monkeys 1995 Mystery Sci...
3,47,Seven (a.k.a. Se7en) (1995),Mystery Thriller,Seven aka Se7en 1995 Mystery Thriller
4,50,"Usual Suspects, The (1995)",Crime Mystery Thriller,Usual Suspects The 1995 Crime Mystery Thriller


In [9]:
len(df_clean)

12277

In [10]:
df_clean = df_clean.reset_index(drop=True)
df_clean.head()

,itemID,title,genres,cleaned_text
0,2,Jumanji (1995),Adventure Children Fantasy,Jumanji 1995 Adventure Children Fantasy
1,29,"City of Lost Children, The (Cité des enfants p...",Adventure Drama Fantasy Mystery Sci-Fi,City of Lost Children The Cité des enfants per...
2,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery Sci-Fi Thriller,Twelve Monkeys aka 12 Monkeys 1995 Mystery Sci...
3,47,Seven (a.k.a. Se7en) (1995),Mystery Thriller,Seven aka Se7en 1995 Mystery Thriller
4,50,"Usual Suspects, The (1995)",Crime Mystery Thriller,Usual Suspects The 1995 Crime Mystery Thriller


In [11]:
header = {
    "col_user": "userID",
    "col_item": "itemID",
    "col_rating": "rating",
    "col_timestamp": "timestamp",
    "col_title": "title",
    "col_genres": "genres",
    "col_year": "year",
    "col_prediction": "prediction",
}

In [12]:
train, test = python_stratified_split(
    data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42
)

In [13]:
train = recommender.clean_dataframe(train, cols_to_clean, clean_col)
train.head()

,userID,itemID,rating,timestamp,title,genres,cleaned_text
72,1,2140,4.0,2005-04-02 23:48:25,"Dark Crystal, The (1982)",Adventure Fantasy,Dark Crystal The 1982 Adventure Fantasy
128,1,4915,3.0,2005-04-02 23:54:36,"Beastmaster, The (1982)",Action Adventure Fantasy,Beastmaster The 1982 Action Adventure Fantasy
171,1,8636,4.5,2005-04-02 23:44:53,Spider-Man 2 (2004),Action Adventure Sci-Fi IMAX,SpiderMan 2 2004 Action Adventure SciFi IMAX
10,1,293,4.0,2005-04-02 23:31:43,Léon: The Professional (a.k.a. The Professiona...,Action Crime Drama Thriller,Léon The Professional aka The Professional Léo...
98,1,3000,3.5,2005-04-02 23:29:29,Princess Mononoke (Mononoke-hime) (1997),Action Adventure Animation Drama Fantasy,Princess Mononoke Mononokehime 1997 Action Adv...


Let's also tokenize the cleaned text for use in the TF-IDF model. The tokens are stored within our TfidfRecommender object.

In [14]:
# Tokenize text with tokenization_method specified in class instantiation
tf, vectors_tokenized = recommender.tokenize_text(df_clean, text_col="cleaned_text")

### 6. Recommend articles using TF-IDF
Let's now fit the recommender model to the processed data (tokens) and retrieve the top k recommended articles.

When creating our object, we specified k=5 so the `recommend_top_k_items` function will return the top 5 recommendations for each public domain article.

In [15]:
# Fit the TF-IDF vectorizer
recommender.fit(tf, vectors_tokenized)
tokens = recommender.get_tokens()
print(list(tokens.keys())[:10])


['ju', 'man', 'ji', '1995', 'adventure', 'children', 'fantasy', 'ju man', 'man ji', 'ji 1995']


In [16]:
len(tokens)

78314

In [17]:
top_k_items = recommender.recommend_top_k_items(df_clean, k=5)
top_k_items.sort_values("rec_score", ascending=False)

,itemID,rec_rank,rec_score,rec_itemID
2166,252,2,1.000000,289
60590,78116,1,1.000000,78116
9446,289,2,1.000000,289
40415,6293,1,1.000000,6293
34130,33838,1,1.000000,33838
...,...,...,...,...
49007,6226,3,0.050660,6781
49008,6226,4,0.050313,92427
49009,6226,5,0.049677,27563
60168,66019,4,0.046999,2714


In [18]:
merged_df = data.merge(top_k_items, on='itemID', how='inner')
merged_df['prediction'] = merged_df['rating'] * merged_df['rec_score']
top_k = merged_df[['userID', 'rec_itemID', 'prediction']]
top_k.rename(columns={'rec_itemID': 'itemID'}, inplace=True)
top_k.sort_values(["userID", "prediction"], ascending=[True, False]).head()

C:\Users\julia\AppData\Local\Temp\ipykernel_11420\892789179.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_k.rename(columns={'rec_itemID': 'itemID'}, inplace=True)


,userID,itemID,prediction
450,1,3795,2.482931
345,1,8610,2.425761
615,1,107997,2.146685
240,1,6461,2.138972
595,1,4886,2.026795


In [19]:
filtered_top_k = top_k.merge(train, on=["userID", "itemID"], how="left", indicator=True)
filtered_top_k = filtered_top_k[filtered_top_k["_merge"] == "left_only"].drop(columns=["_merge"])
filtered_top_k = filtered_top_k[["userID", "itemID", "prediction"]]
filtered_top_k.head()

,userID,itemID,prediction
0,1,31447,1.160394
1,1,2103,1.030680
2,1,56,0.991387
3,1,60,0.981581
4,1,158,0.979672


In [20]:
idx = filtered_top_k.groupby("userID")["prediction"].idxmax()
top = filtered_top_k.loc[idx]
top.head()

,userID,itemID,prediction
450,1,3795,2.482931
1105,2,6667,2.644084
2070,3,519,2.716976
2250,4,836,2.419135
2530,5,635,3.062231


In [21]:
args = [test, top_k]
kwargs = dict(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_prediction="prediction",
    relevancy_method="top_k",
    k=TOP_K
)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg_at_k = ndcg_at_k(*args, **kwargs)
eval_precision_at_k = precision_at_k(*args, col_user="userID", col_item="itemID", col_rating="rating", col_prediction="prediction", k=TOP_K)
eval_recall_at_k = recall_at_k(*args, col_user="userID", col_item="itemID", col_rating="rating", col_prediction="prediction", k=TOP_K)

In [22]:
args1 = [test, top]
kwargs1 = dict(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_prediction="prediction",
    relevancy_method="top_k",
    k=1
)
eval_ndcg = ndcg_at_k(*args1, **kwargs1)
eval_precision = precision_at_k(*args, col_user="userID", col_item="itemID", col_rating="rating", col_prediction="prediction", k=1)
eval_recall = recall_at_k(*args, col_user="userID", col_item="itemID", col_rating="rating", col_prediction="prediction", k=1)

In [23]:
eval_mae = mae(test, top_k)
eval_rmse = rmse(test, top_k)

In [24]:
# eval_novelty = novelty(train, top)
# eval_historical_item_novelty = historical_item_novelty(train, top)
# eval_user_item_serendipity = user_item_serendipity(train, top)
# eval_user_serendipity = user_serendipity(train, top)
# eval_serendipity = serendipity(train, top)
# eval_catalog_coverage = catalog_coverage(train, top)
# eval_distributional_coverage = distributional_coverage(train, top)

In [25]:
# eval_f1 = f1(test, top)
# eval_mrr = mrr(test, top)
# eval_accuracy = accuracy(test, top)
# eval_user_coverage = user_coverage(test, top)
# eval_item_coverage = item_coverage(test, top)

In [26]:
print("Precision:\t%f" % eval_precision,
      "Precision@K:\t%f" % eval_precision_at_k,
      "Recall:\t%f" % eval_recall,
      "Recall@K:\t%f" % eval_recall_at_k,
      "MAE:\t%f" % eval_mae,
      "RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      # "Novelty:\t%f" % eval_novelty,
      # "Serendipity:\t%f" % eval_serendipity,
      # "Catalog coverage:\t%f" % eval_catalog_coverage,
      # "Distributional coverage:\t%f" % eval_distributional_coverage,
      sep='\n')

Precision:	0.022229
Precision@K:	0.203616
Recall:	0.000902
Recall@K:	0.009068
MAE:	2.486453
RMSE:	2.684313
NDCG:	0.030528


In [27]:
# print("F1:\t%f" % eval_f1,
#       "Accuracy:\t%f" % eval_accuracy,
#       "User covarage:\t%f" % eval_user_coverage,
#       "Item coverage:\t%f" % eval_item_coverage,
#       sep='\n')

In [28]:
# eval_historical_item_novelty

In [29]:
# eval_user_item_serendipity

In [30]:
# eval_user_serendipity